In [1]:
import requests
import pandas as pd

In [2]:
!pip install sodapy

In [3]:

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.sfgov.org", None)

results = client.get("wg3w-h783", limit=5000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df.head()
results_df.isnull().sum()
# crime_counts = results_df.groupby(['latitude', 'longitude']).size().reset_index(name='crime_count')
# crime.isnull().sum()
results_df = results_df.drop([':@computed_region_26cr_cadq',
 ':@computed_region_qgnn_b9vv',
 ':@computed_region_jwn9_ihcz',
 ':@computed_region_h4ep_8xdi',
 ':@computed_region_nqbw_i6c3',
 ':@computed_region_n4xg_c4py',
 ':@computed_region_jg9y_a9du'], axis=1)
results_df = results_df.dropna(subset=['longitude', 'latitude'])
# results_df.to_excel("output.xlsx")
# results_df.add(crime_counts)


In [4]:
# prompt: use results_df and convert it to json format

results_df.to_json("output.json", orient="records")


In [5]:
import folium
map = folium.Map(
 location=[37.753244, -122.441719],
 zoom_start=12.5
)

In [6]:
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster(
 name="Crimes by Marker",
 overlay=True,
 control=True
)

In [7]:
lat = results_df['latitude'].tolist()
lng = results_df['longitude'].tolist()
# crime_counts = crime_counts['crime_count'].tolist()
offense = results_df['incident_category'].tolist()
locations = list(zip(lat, lng))
offense = results_df['incident_category'].tolist()
# results_df.insert(10, "Crime Count", crime_counts)
for i in range(len(lat)):
 location = lat[i], lng[i]
 crime_type = offense[i]
#  crime_count = crime_counts[i]
 html = '''<b> Type of Crime: </b> {}<br>
 Latitude: {}<br>
 Longitude:{}<br>
'''.format(crime_type, location[0], location[1])
 iframe = folium.IFrame(html, width=200, height=200)
 popup = folium.Popup(iframe, max_width=200)
 marker = folium.Marker(location= location,popup=popup)
 marker_cluster.add_child(marker)
marker_cluster.add_to(map)

# print(crime_counts)
results_df

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,report_type_code,...,filed_online,cad_number,intersection,cnn,analysis_neighborhood,supervisor_district,supervisor_district_2012,latitude,longitude,point
9,2023-03-11T14:00:00.000,2023-03-11T00:00:00.000,14:00,2023,Saturday,2023-03-15T11:21:00.000,125431804134,1254318,230182844,II,...,NaN,230741133,STANYAN ST \ HAYES ST,26446000,Golden Gate Park,1,1,37.772895177200766,-122.45428511766733,"{'type': 'Point', 'coordinates': [-122.4542851..."
11,2022-06-27T12:00:00.000,2022-06-27T00:00:00.000,12:00,2022,Monday,2023-03-15T17:20:00.000,125439371000,1254393,230184129,II,...,NaN,230742480,GEARY ST \ POWELL ST,24903000,Financial District/South Beach,3,3,37.78735926098589,-122.40822672700406,"{'type': 'Point', 'coordinates': [-122.4082267..."
13,2023-03-16T17:30:00.000,2023-03-16T00:00:00.000,17:30,2023,Thursday,2023-03-16T18:02:00.000,125482604134,1254826,230187101,II,...,NaN,230752550,18TH ST \ DE HARO ST,23743000,Potrero Hill,10,10,37.76228996810526,-122.40132418490647,"{'type': 'Point', 'coordinates': [-122.4013241..."
33,2023-03-21T15:50:00.000,2023-03-21T00:00:00.000,15:50,2023,Tuesday,2023-03-21T16:01:00.000,125656351040,1256563,230199764,II,...,NaN,230802198,POST ST \ LARKIN ST,25167000,Tenderloin,3,6,37.787037946181535,-122.41827098126804,"{'type': 'Point', 'coordinates': [-122.4182709..."
61,2021-08-22T09:40:00.000,2021-08-22T00:00:00.000,09:40,2021,Sunday,2021-08-22T09:40:00.000,106267662071,1062676,210537297,II,...,NaN,212340900,LAGUNA ST \ PACIFIC AVE,26569000,Pacific Heights,2,2,37.79397724418211,-122.42980398313114,"{'type': 'Point', 'coordinates': [-122.4298039..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-02-10T20:00:00.000,2023-02-10T00:00:00.000,20:00,2023,Friday,2023-03-28T15:49:00.000,125885971000,1258859,230216346,II,...,NaN,230872016,POLK ST \ FERN ST,25209000,Nob Hill,3,3,37.788222699672026,-122.42021043633935,"{'type': 'Point', 'coordinates': [-122.4202104..."
4996,2023-03-23T16:15:00.000,2023-03-23T00:00:00.000,16:15,2023,Thursday,2023-03-28T16:57:00.000,125889816220,1258898,230216578,II,...,NaN,230872279,JESSIE ST \ 09TH ST,24416000,South of Market,6,6,37.77671770313698,-122.41532057072628,"{'type': 'Point', 'coordinates': [-122.4153205..."
4997,2023-03-28T10:59:00.000,2023-03-28T00:00:00.000,10:59,2023,Tuesday,2023-03-28T10:59:00.000,125879907046,1258799,230194225,VS,...,NaN,230870794,JERROLD AVE \ PHELPS ST,20672000,Bayview Hunters Point,10,10,37.73985319897475,-122.39126842523832,"{'type': 'Point', 'coordinates': [-122.3912684..."
4998,2023-03-28T14:40:00.000,2023-03-28T00:00:00.000,14:40,2023,Tuesday,2023-03-28T21:30:00.000,125894909320,1258949,230217140,II,...,NaN,230873070,PORTOLA DR \ CLAREMONT BLVD \ PORTAL PATH,22921000,West of Twin Peaks,7,7,37.73968664480658,-122.46523744291983,"{'type': 'Point', 'coordinates': [-122.4652374..."


In [8]:
# prompt: convert json_datat to dictionary
import json
json_data = results_df.to_json(orient='split')


In [9]:
from folium.plugins import HeatMap
HeatMap(
 data=list(zip(lat, lng)),
 name="Crimes by Heatmap").add_to(map)
folium.LayerControl().add_to(map)
# map

In [10]:
!pip install firebase_admin
!pip show firebase-admin

Name: firebase-admin
Version: 5.3.0
Summary: Firebase Admin Python SDK
Home-page: https://firebase.google.com/docs/admin/setup/
Author: Firebase
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: cachecontrol, google-api-core, google-api-python-client, google-cloud-firestore, google-cloud-storage
Required-by: 


In [11]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials
from firebase_admin import db
database_url = "https://safespot-46b42-default-rtdb.firebaseio.com"
cred = credentials.Certificate("/content/drive/MyDrive/SafeSpot/safespot-46b42-firebase-adminsdk-i6krf-085360ebfc.json")
if not firebase_admin._apps:
    # Initialize the Firebase Admin SDK
    firebase_admin.initialize_app(cred, {"databaseURL": database_url})
else:
    print("Firebase app is already initialized.")

In [12]:
db = firestore.client()
doc_ref = db.collection("SafeSpotData").document("H9gXFFnLGXog28m001Ts")
with open("output.json", "r") as f:
    data = json.load(f)

# Add the data to Firestore
for record in data:
    db.collection("crimestatus").add(record)

print("Data successfully added to Firestore")

Data successfully added to Firestore
